In [7]:
import numpy as np
from Crypto.Cipher import ChaCha20
import math
import os

# Helper functions
## Read the captured data

In [2]:
NONCE_LEN_BYTES = 12
TRACE_RANDOM_CNT = None
CHUNK_SIZE = None
CHUNKS_CNT = None
LAST_CHUNK_SIZE = None

NONCES = None
NONCE_LEN_BYTES = 12

PLAINTEXTS = None
PLAINTEXT_LEN_BYTES = 64

CIPHERTEXTS = None
CIPHERTEXT_LEN_BYTES = 64 


In [3]:
def read_info(folder):
    global CHUNK_SIZE, LAST_CHUNK_SIZE
    global CHUNKS_CNT
    with open(f"{folder}/info.txt", 'r') as file:
        global TRACE_CNT
        global TRACE_LEN
        global TRACE_RANDOM_CNT
        TRACE_CNT = int(file.readline())
        TRACE_RANDOM_CNT = TRACE_CNT
        CHUNK_SIZE = int(file.readline())
        TRACE_LEN = int(file.readline())
        
    CHUNKS_CNT = math.ceil(TRACE_CNT / CHUNK_SIZE)
    LAST_CHUNK_SIZE = TRACE_CNT - (CHUNKS_CNT - 1)* CHUNK_SIZE
    print(f"TRACE_CNT = {TRACE_CNT}")   
    print(f"CHUNK_SIZE = {CHUNK_SIZE}")   
    print(f"LAST_CHUNK_SIZE = {LAST_CHUNK_SIZE}")   
    print(f"CHUNKS_CNT = {CHUNKS_CNT}")   
    print(f"TRACE_RANDOM_CNT = {TRACE_RANDOM_CNT}")   
    print(f"TRACE_LEN = {TRACE_LEN}")

In [9]:
NONCES = None
def read_nonces(FOLDER):
    global NONCES
    global CHUNK_SIZE, LAST_CHUNK_SIZE
    global CHUNKS_CNT
    global TRACE_CNT
    # Calculate the number of chunks
    
    nonces_list = []
    for chunk_index in range(CHUNKS_CNT):
        chunk_folder = os.path.join(FOLDER, f"chunk_{chunk_index}")
        chunk_file = os.path.join(chunk_folder, "nonces_random.bin")
        
        if os.path.exists(chunk_file):
            with open(chunk_file, 'rb') as file:
                byte_array = file.read()
            
            if chunk_index != CHUNKS_CNT-1:
                chunk = np.frombuffer(byte_array, dtype=np.uint8).reshape((CHUNK_SIZE, NONCE_LEN_BYTES))
            else:
                chunk = np.frombuffer(byte_array, dtype=np.uint8).reshape((LAST_CHUNK_SIZE, NONCE_LEN_BYTES))

            nonces_list.append(chunk)
        else:
            print(f"Chunk file {chunk_file} does not exist.")
    
    # Concatenate all chunks
    NONCES = np.vstack(nonces_list)
    
    nonce = NONCES[0]
    # Convert each byte to its hexadecimal representation and join them
    hex_representation = ''.join(f'{byte:02x}' for byte in nonce[:12])
    # Print the result
    print(f"Nonce[0] = {hex_representation}")

In [10]:
CIPHERTEXTS = None
def read_ciphertexts(folder):
    global CIPHERTEXTS
    global CHUNK_SIZE, LAST_CHUNK_SIZE
    global CHUNKS_CNT
    global TRACE_CNT
    # Calculate the number of chunks
    
    ciphertexts_list = []
    for chunk_index in range(CHUNKS_CNT):
        chunk_folder = os.path.join(folder, f"chunk_{chunk_index}")
        chunk_file = os.path.join(chunk_folder, "ciphertexts_random.bin")
        
        if os.path.exists(chunk_file):
            with open(chunk_file, 'rb') as file:
                byte_array = file.read()
            
            if chunk_index != CHUNKS_CNT-1:
                chunk_ciphertexts = np.frombuffer(byte_array, dtype=np.uint8).reshape((CHUNK_SIZE, CIPHERTEXT_LEN_BYTES))
            else:
                chunk_ciphertexts = np.frombuffer(byte_array, dtype=np.uint8).reshape((LAST_CHUNK_SIZE, CIPHERTEXT_LEN_BYTES))

            ciphertexts_list.append(chunk_ciphertexts)
        else:
            print(f"Chunk file {chunk_file} does not exist.")
    
    # Concatenate all chunks
    CIPHERTEXTS = np.vstack(ciphertexts_list)
        
    ciphertext = CIPHERTEXTS[0]
    # Convert each byte to its hexadecimal representation and join them
    hex_representation = ''.join(f'{byte:02x}' for byte in ciphertext[:64])
    # Print the result
    print(f"CT[0] = {hex_representation}")

In [11]:
PLAINTEXTS = None
def read_plaintexts(folder):
    global PLAINTEXTS
    global CHUNK_SIZE, LAST_CHUNK_SIZE
    global CHUNKS_CNT
    global TRACE_CNT
    # Calculate the number of chunks
    
    pts_list = []
    for chunk_index in range(CHUNKS_CNT):
        chunk_folder = os.path.join(folder, f"chunk_{chunk_index}")
        chunk_file = os.path.join(chunk_folder, "plaintexts_random.bin")
        
        if os.path.exists(chunk_file):
            with open(chunk_file, 'rb') as file:
                byte_array = file.read()
            
            if chunk_index != CHUNKS_CNT-1:
                chunk_pts = np.frombuffer(byte_array, dtype=np.uint8).reshape((CHUNK_SIZE, PLAINTEXT_LEN_BYTES))
            else:
                chunk_pts = np.frombuffer(byte_array, dtype=np.uint8).reshape((LAST_CHUNK_SIZE, PLAINTEXT_LEN_BYTES))

            pts_list.append(chunk_pts)
        else:
            print(f"Chunk file {chunk_file} does not exist.")
    
    # Concatenate all chunks
    PLAINTEXTS = np.vstack(pts_list)
    
    # Desired shape
    desired_shape = (TRACE_CNT, PLAINTEXT_LEN_BYTES)
        
    pt = PLAINTEXTS[0]
    # Convert each byte to its hexadecimal representation and join them
    hex_representation = ''.join(f'{byte:02x}' for byte in pt[:64])
    # Print the result
    print(f"PT[0] = {hex_representation}")

## Function to perform the testing

In [14]:
def verify_encryptions():
    key = CORRECT_KEY.tobytes()
    # Loop through each value and check encryption/decryption
    for i in range(TRACE_CNT):
        # Extract nonce, plaintext, and expected ciphertext
        nonce_i = NONCES[i].tobytes()
        pt_i = PLAINTEXTS[i].tobytes()
        ct_i = CIPHERTEXTS[i].tobytes()

        # Encrypt the plaintext
        cipher = ChaCha20.new(key=key, nonce=nonce_i)
        generated_ct = cipher.encrypt(pt_i)

        # Check if the generated ciphertext matches the expected ciphertext
        if generated_ct != ct_i:
            print(f"Mismatch found at index {i} during encryption")
            print(f'KEY    - {key.hex()}')
            print(f'NONCE  - {nonce_i.hex()}')
            print(f'PT     - {pt_i.hex()}')
            print(f'RESULTS:')
            print(f'Python - {generated_ct.hex()}')
            print(f'MCU    - {ct_i.hex()}')
    #         break

        # Decrypt the ciphertext
        cipher = ChaCha20.new(key=key, nonce=nonce_i)
        decrypted_pt = cipher.decrypt(ct_i)

        # Check if the decrypted plaintext matches the original plaintext
        if decrypted_pt != pt_i:
            print(f"Mismatch found at index {i} during decryption")
            print(f'KEY    - {key.hex()}')
            print(f'NONCE  - {nonce_i.hex()}')
            print(f'PT     - {pt_i.hex()}')
            print(f'CT     - {ct_i.hex()}')
            print(f'RESULTS:')
            print(f'Python - {decrypted_pt.hex()}')
            break
    else:
        print("All values matched successfully.")

# Verify the data

## XMEGA

In [5]:
FOLDER = "ChaCha-100-000-Random-Nonce-XMEGA"

In [8]:
read_info(FOLDER)

TRACE_CNT = 100000
CHUNK_SIZE = 10000
LAST_CHUNK_SIZE = 10000
CHUNKS_CNT = 10
TRACE_RANDOM_CNT = 100000
TRACE_LEN = 21304


In [12]:
read_nonces(FOLDER)
read_plaintexts(FOLDER)
read_ciphertexts(FOLDER)

Nonce[0] = 25cc4792cfd34a0418243218
PT[0] = 918d99c3da107c664a58f688c228fdb4fb0a4b8b8ca70a1a6399f8f504eb4db607846b6c437a2ed9634e95e3c723d713e3d8ff8855ef58d4219c41b2335840fe
CT[0] = 98971d085927a07bf057543c5458919f545223dc3b78ea51d563db098119a44161fa6c8823dca6870c41fbc0087432650e1c3f0c2b8e0a03a9a3fc40f2f18202


In [13]:
#load the correct key
CORRECT_KEY = np.fromfile(f"{FOLDER}/key.bin", dtype=np.uint8)
hex_key = ''.join(f'{byte:02x}' for byte in CORRECT_KEY)
print(f"Correct Key: {hex_key}")

Correct Key: 3afa3f58fbc0f2bc68e5ac304bb50e43833971578b132a87d9621f51b1f4e6e0


In [15]:
verify_encryptions()

All values matched successfully.


## STM

In [16]:
FOLDER = "ChaCha-100-000-Random-Nonce-STM"

In [17]:
read_info(FOLDER)

TRACE_CNT = 100000
CHUNK_SIZE = 10000
LAST_CHUNK_SIZE = 10000
CHUNKS_CNT = 10
TRACE_RANDOM_CNT = 100000
TRACE_LEN = 9168


In [18]:
read_nonces(FOLDER)
read_plaintexts(FOLDER)
read_ciphertexts(FOLDER)

Nonce[0] = fcc1c7ef50f1888e8847a966
PT[0] = 09e2271a54307e043b82f1a859dae46ab66b81e2f19fd24a66c05cc85da78b7c991355fbd81ac7cfed82df949dbe11216f11cac6f9d1a0cd30e3e603411d0cb4
CT[0] = 03dc065b81abb512b003f85569a943e3b323c74b2161576808986301e41c5f9255fc6c339d9157800f053045a726ed3676875fb9edf525e6ee043e910585e96e


In [19]:
#load the correct key
CORRECT_KEY = np.fromfile(f"{FOLDER}/key.bin", dtype=np.uint8)
hex_key = ''.join(f'{byte:02x}' for byte in CORRECT_KEY)
print(f"Correct Key: {hex_key}")

Correct Key: db48fa60a61f68bf31b12ec9344e68d8f5782105fa4b67eebf67c46345f5a402


In [20]:
verify_encryptions()

All values matched successfully.
